In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [5]:
train,test = tfds.load(name='imdb_reviews',split = ('train','test'),as_supervised=True)

In [8]:
train_examples,train_labels = next(iter(train.batch(10)))
train_examples

<tf.Tensor: id=123, shape=(10,), dtype=string, numpy=
array([b'Brian De Palma\'s undeniable virtuosity can\'t really camouflage the fact that his plot here is a thinly disguised "Psycho" carbon copy, but he does provide a genuinely terrifying climax. His "Blow Out", made the next year, was an improvement.',
       b"This was soul-provoking! I am an Iranian, and living in th 21st century, I didn't know that such big tribes have been living in such conditions at the time of my grandfather!<br /><br />You see that today, or even in 1925, on one side of the world a lady or a baby could have everything served for him or her clean and on-demand, but here 80 years ago, people ventured their life to go to somewhere with more grass. It's really interesting that these Persians bear those difficulties to find pasture for their sheep, but they lose many the sheep on their way.<br /><br />I praise the Americans who accompanied this tribe, they were as tough as Bakhtiari people.",
       b"Being a f

In [9]:
train_labels

<tf.Tensor: id=124, shape=(10,), dtype=int64, numpy=array([1, 1, 0, 1, 1, 0, 1, 1, 0, 1], dtype=int64)>

In [11]:
embedding="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding,input_shape=[],dtype = tf.string,trainable = True)
hub_layer(train_examples[:3])

<tf.Tensor: id=304, shape=(3, 20), dtype=float32, numpy=
array([[ 0.71892303, -2.0958326 ,  2.481605  , -0.24909717, -1.6124796 ,
        -1.0279148 , -1.7041445 ,  0.05924067,  2.0719886 , -1.3690318 ,
        -1.2510301 ,  0.26544   ,  0.4398559 , -0.23960635, -0.4651044 ,
         0.78264546,  1.3775747 , -0.92917114, -1.3016398 , -0.9623547 ],
       [ 2.2488396 , -1.4537774 ,  1.97866   , -0.73290443, -2.2218246 ,
        -4.054772  , -1.6585274 ,  1.8872426 ,  1.8315402 ,  0.45302168,
        -0.50105846,  1.3690453 , -1.9862492 ,  0.4043505 , -5.337918  ,
         1.5515825 ,  3.6844683 , -3.3751655 , -3.4578393 , -1.1764543 ],
       [ 1.7955422 , -3.6056843 ,  2.6997256 , -0.60008395, -4.6546526 ,
        -3.7613652 , -3.0317647 ,  0.04494802,  2.3193102 , -1.2068036 ,
        -1.9191952 ,  2.5517764 , -0.35324278,  0.01286946, -4.1438556 ,
         3.4991353 ,  3.2947102 , -2.3533776 , -3.3100967 , -1.0114092 ]],
      dtype=float32)>

In [12]:
model=tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


The layers are stacked sequentially to build the classifier:

The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its embedding vector. The pre-trained text embedding model that we are using (google/tf2-preview/gnews-swivel-20dim/1) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: (num_examples, embedding_dimension).
This fixed-length output vector is piped through a fully-connected (Dense) layer with 16 hidden units.
The last layer is densely connected with a single output node.

In [13]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [16]:
history = model.fit(train.shuffle(10000).batch(512),epochs = 20,verbose=1)


Epoch 1/20
49/49 [==============================].1288 - accuracy: 0.94 - 2s 914ms/step - loss: 0.1168 - accuracy: 0.955 - 2s 683ms/step - loss: 0.1129 - accuracy: 0.957 - 2s 564ms/step - loss: 0.1082 - accuracy: 0.961 - 2s 492ms/step - loss: 0.1100 - accuracy: 0.962 - 3s 444ms/step - loss: 0.1091 - accuracy: 0.963 - 3s 402ms/step - loss: 0.1113 - accuracy: 0.962 - 3s 373ms/step - loss: 0.1114 - accuracy: 0.961 - 3s 350ms/step - loss: 0.1097 - accuracy: 0.962 - 3s 331ms/step - loss: 0.1090 - accuracy: 0.962 - 3s 317ms/step - loss: 0.1101 - accuracy: 0.962 - 4s 304ms/step - loss: 0.1089 - accuracy: 0.963 - 4s 294ms/step - loss: 0.1088 - accuracy: 0.964 - 4s 285ms/step - loss: 0.1079 - accuracy: 0.965 - 4s 278ms/step - loss: 0.1090 - accuracy: 0.964 - 4s 270ms/step - loss: 0.1072 - accuracy: 0.965 - 4s 264ms/step - loss: 0.1071 - accuracy: 0.965 - 5s 259ms/step - loss: 0.1061 - accuracy: 0.965 - 5s 253ms/step - loss: 0.1055 - accuracy: 0.965 - 5s 250ms/step - loss: 0.1058 - accuracy: 0.9

49/49 [==============================] - ETA: 1:10 - loss: 0.0773 - accuracy: 0.98 - ETA: 38s - loss: 0.0794 - accuracy: 0.9834 - ETA: 27s - loss: 0.0794 - accuracy: 0.982 - ETA: 22s - loss: 0.0824 - accuracy: 0.979 - ETA: 18s - loss: 0.0864 - accuracy: 0.976 - ETA: 16s - loss: 0.0856 - accuracy: 0.975 - ETA: 14s - loss: 0.0829 - accuracy: 0.977 - ETA: 13s - loss: 0.0898 - accuracy: 0.971 - ETA: 12s - loss: 0.0893 - accuracy: 0.971 - ETA: 11s - loss: 0.0887 - accuracy: 0.971 - ETA: 10s - loss: 0.0883 - accuracy: 0.971 - ETA: 9s - loss: 0.0888 - accuracy: 0.970 - ETA: 9s - loss: 0.0879 - accuracy: 0.97 - ETA: 8s - loss: 0.0887 - accuracy: 0.97 - ETA: 8s - loss: 0.0889 - accuracy: 0.97 - ETA: 7s - loss: 0.0893 - accuracy: 0.97 - ETA: 7s - loss: 0.0896 - accuracy: 0.97 - ETA: 6s - loss: 0.0906 - accuracy: 0.97 - ETA: 6s - loss: 0.0902 - accuracy: 0.97 - ETA: 6s - loss: 0.0901 - accuracy: 0.97 - ETA: 5s - loss: 0.0907 - accuracy: 0.97 - ETA: 5s - loss: 0.0900 - accuracy: 0.97 - ETA: 5s - l

49/49 [==============================] - ETA: 41s - loss: 0.0726 - accuracy: 0.974 - ETA: 23s - loss: 0.0696 - accuracy: 0.976 - ETA: 17s - loss: 0.0667 - accuracy: 0.980 - ETA: 15s - loss: 0.0681 - accuracy: 0.980 - ETA: 13s - loss: 0.0696 - accuracy: 0.980 - ETA: 11s - loss: 0.0695 - accuracy: 0.980 - ETA: 10s - loss: 0.0717 - accuracy: 0.980 - ETA: 9s - loss: 0.0711 - accuracy: 0.980 - ETA: 8s - loss: 0.0698 - accuracy: 0.98 - ETA: 8s - loss: 0.0689 - accuracy: 0.98 - ETA: 7s - loss: 0.0713 - accuracy: 0.97 - ETA: 7s - loss: 0.0716 - accuracy: 0.97 - ETA: 6s - loss: 0.0721 - accuracy: 0.97 - ETA: 6s - loss: 0.0730 - accuracy: 0.97 - ETA: 5s - loss: 0.0723 - accuracy: 0.97 - ETA: 5s - loss: 0.0725 - accuracy: 0.97 - ETA: 5s - loss: 0.0726 - accuracy: 0.97 - ETA: 5s - loss: 0.0726 - accuracy: 0.97 - ETA: 4s - loss: 0.0728 - accuracy: 0.97 - ETA: 4s - loss: 0.0732 - accuracy: 0.97 - ETA: 4s - loss: 0.0728 - accuracy: 0.97 - ETA: 4s - loss: 0.0728 - accuracy: 0.97 - ETA: 4s - loss: 0.07

49/49 [==============================] - ETA: 1:07 - loss: 0.0670 - accuracy: 0.97 - ETA: 37s - loss: 0.0581 - accuracy: 0.9824 - ETA: 26s - loss: 0.0589 - accuracy: 0.981 - ETA: 21s - loss: 0.0642 - accuracy: 0.982 - ETA: 18s - loss: 0.0640 - accuracy: 0.981 - ETA: 16s - loss: 0.0641 - accuracy: 0.982 - ETA: 14s - loss: 0.0624 - accuracy: 0.983 - ETA: 13s - loss: 0.0616 - accuracy: 0.983 - ETA: 12s - loss: 0.0614 - accuracy: 0.983 - ETA: 11s - loss: 0.0607 - accuracy: 0.983 - ETA: 10s - loss: 0.0608 - accuracy: 0.984 - ETA: 9s - loss: 0.0607 - accuracy: 0.983 - ETA: 9s - loss: 0.0606 - accuracy: 0.98 - ETA: 8s - loss: 0.0602 - accuracy: 0.98 - ETA: 8s - loss: 0.0589 - accuracy: 0.98 - ETA: 7s - loss: 0.0586 - accuracy: 0.98 - ETA: 7s - loss: 0.0584 - accuracy: 0.98 - ETA: 6s - loss: 0.0592 - accuracy: 0.98 - ETA: 6s - loss: 0.0592 - accuracy: 0.98 - ETA: 6s - loss: 0.0592 - accuracy: 0.98 - ETA: 5s - loss: 0.0586 - accuracy: 0.98 - ETA: 5s - loss: 0.0584 - accuracy: 0.98 - ETA: 5s - l

49/49 [==============================] - ETA: 1:12 - loss: 0.0429 - accuracy: 0.98 - ETA: 40s - loss: 0.0459 - accuracy: 0.9883 - ETA: 29s - loss: 0.0487 - accuracy: 0.988 - ETA: 24s - loss: 0.0484 - accuracy: 0.987 - ETA: 20s - loss: 0.0476 - accuracy: 0.986 - ETA: 18s - loss: 0.0484 - accuracy: 0.986 - ETA: 16s - loss: 0.0490 - accuracy: 0.985 - ETA: 14s - loss: 0.0499 - accuracy: 0.985 - ETA: 13s - loss: 0.0511 - accuracy: 0.985 - ETA: 12s - loss: 0.0512 - accuracy: 0.985 - ETA: 11s - loss: 0.0505 - accuracy: 0.985 - ETA: 10s - loss: 0.0509 - accuracy: 0.985 - ETA: 9s - loss: 0.0515 - accuracy: 0.985 - ETA: 9s - loss: 0.0511 - accuracy: 0.98 - ETA: 8s - loss: 0.0509 - accuracy: 0.98 - ETA: 8s - loss: 0.0500 - accuracy: 0.98 - ETA: 7s - loss: 0.0501 - accuracy: 0.98 - ETA: 7s - loss: 0.0495 - accuracy: 0.98 - ETA: 7s - loss: 0.0493 - accuracy: 0.98 - ETA: 6s - loss: 0.0488 - accuracy: 0.98 - ETA: 6s - loss: 0.0484 - accuracy: 0.98 - ETA: 6s - loss: 0.0484 - accuracy: 0.98 - ETA: 5s -

49/49 [==============================] - ETA: 1:18 - loss: 0.0329 - accuracy: 0.99 - ETA: 43s - loss: 0.0376 - accuracy: 0.9902 - ETA: 31s - loss: 0.0383 - accuracy: 0.988 - ETA: 25s - loss: 0.0396 - accuracy: 0.989 - ETA: 21s - loss: 0.0385 - accuracy: 0.991 - ETA: 19s - loss: 0.0379 - accuracy: 0.991 - ETA: 16s - loss: 0.0384 - accuracy: 0.991 - ETA: 15s - loss: 0.0382 - accuracy: 0.991 - ETA: 14s - loss: 0.0396 - accuracy: 0.991 - ETA: 12s - loss: 0.0391 - accuracy: 0.991 - ETA: 12s - loss: 0.0391 - accuracy: 0.991 - ETA: 11s - loss: 0.0394 - accuracy: 0.991 - ETA: 10s - loss: 0.0397 - accuracy: 0.991 - ETA: 10s - loss: 0.0391 - accuracy: 0.991 - ETA: 9s - loss: 0.0390 - accuracy: 0.991 - ETA: 8s - loss: 0.0394 - accuracy: 0.99 - ETA: 8s - loss: 0.0398 - accuracy: 0.99 - ETA: 8s - loss: 0.0400 - accuracy: 0.99 - ETA: 7s - loss: 0.0397 - accuracy: 0.99 - ETA: 7s - loss: 0.0400 - accuracy: 0.99 - ETA: 6s - loss: 0.0397 - accuracy: 0.99 - ETA: 6s - loss: 0.0398 - accuracy: 0.99 - ETA: 

49/49 [==============================] - ETA: 1:13 - loss: 0.0429 - accuracy: 0.99 - ETA: 41s - loss: 0.0362 - accuracy: 0.9922 - ETA: 30s - loss: 0.0343 - accuracy: 0.992 - ETA: 24s - loss: 0.0322 - accuracy: 0.992 - ETA: 20s - loss: 0.0324 - accuracy: 0.992 - ETA: 18s - loss: 0.0331 - accuracy: 0.992 - ETA: 16s - loss: 0.0331 - accuracy: 0.992 - ETA: 14s - loss: 0.0328 - accuracy: 0.992 - ETA: 13s - loss: 0.0322 - accuracy: 0.993 - ETA: 12s - loss: 0.0312 - accuracy: 0.993 - ETA: 11s - loss: 0.0316 - accuracy: 0.993 - ETA: 10s - loss: 0.0310 - accuracy: 0.993 - ETA: 10s - loss: 0.0315 - accuracy: 0.993 - ETA: 9s - loss: 0.0311 - accuracy: 0.994 - ETA: 8s - loss: 0.0312 - accuracy: 0.99 - ETA: 8s - loss: 0.0314 - accuracy: 0.99 - ETA: 8s - loss: 0.0318 - accuracy: 0.99 - ETA: 7s - loss: 0.0319 - accuracy: 0.99 - ETA: 7s - loss: 0.0315 - accuracy: 0.99 - ETA: 6s - loss: 0.0315 - accuracy: 0.99 - ETA: 6s - loss: 0.0320 - accuracy: 0.99 - ETA: 6s - loss: 0.0320 - accuracy: 0.99 - ETA: 5s

In [17]:
results = model.evaluate(test.batch(512),verbose=2)

49/49 - 7s - loss: 0.5471 - accuracy: 0.8460


In [18]:
for name,value in zip(model.metrics_names,results):
    print("%s: %.3f" %(name,value))

loss: 0.547
accuracy: 0.846
